In [54]:
#Goal is to find the worst performing products
import numpy as np

#import dataset
import pandas as pd
df = pd.read_excel("/home/jessica/senior-design/data/Clean_Data.xlsx", sheet_name="Sheet1")

#remove whitespace from column names for easy access
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('$', 'usd') for c in df.columns]
print(df.columns)

Index(['legacy_system_cd', 'legacy_division_cd', 'legacy_product_cd',
       'legacy_product_desc', 'core_item_flag', 'segment', 'PROD_CAT_1_NAME',
       'PROD_CAT_2_NAME', 'PROD_CAT_3_NAME', 'PROD_CAT_4_NAME',
       'legacy_vendor_cd', 'stocking_flag', 'LEGACY_CUSTOMER_CD',
       'saalfeld_customer_flag', 'national_acct_flag', 'ship-to_zip_code',
       'sales_channel', 'qty_6mos', 'cogs_6mos', 'Sales_6_mos', 'picks_6mos',
       'Margin_%', 'net_OH', 'net_OH_usd', 'pallet_quantity', 'item_poi_days',
       'DIOH', 'Margin_'],
      dtype='object')


In [55]:
segment = 'Packaging'
df = df[df['segment'] == segment]

In [56]:
#Create a dictionary that maps:
#Warehouse (Division) -> Unique products sold at division

warehouses = df.legacy_division_cd.unique()
warehouse_to_prod = {}

for w in warehouses:
    warehouse_to_prod[w] = []

for w, p, sc, sales, costs, qsold, nOH, nOHusd in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.sales_channel, 
                                         df.Sales_6_mos, df.cogs_6mos, df.qty_6mos, df.net_OH, df.net_OH_usd):
    if p not in warehouse_to_prod[w] and sc == "Warehouse" and sales > 0 and costs > 0 and qsold > 0 and nOH > 0 and nOHusd > 0:
        warehouse_to_prod[w].append(p)

# #Print warehouse to number of individual different products stored            
# for w in warehouses:
#     print(w, len(warehouse_to_prod[w]))

In [57]:
#Create a dictionary of warehouse & product -> total sales in a 6month period
wp_to_sales = {}
#Create a dictionary of warehouse & product -> total costs in a 6month period
wp_to_costs = {}
#Create a dictionary of warehouse & product -> total number of picks in a 6month period
wp_to_picks = {}
#Create a dictionary of warehouse & product -> total quantity sold in a 6month period
wp_to_quantity = {}
#Create a dictionary of warehouse & product -> number of customers sold to at that warehouse
wp_to_ncustomers = {}
#Create a dictionary of warehouse & product -> pallet quantity
wp_to_pallet = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_sales[w,p] = []
        wp_to_costs[w,p] = []
        wp_to_picks[w,p] = []
        wp_to_quantity[w,p] = []
        wp_to_ncustomers[w,p] = []

for w, p, s, c, pk, q, cust in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, 
                                   df.Sales_6_mos, df.cogs_6mos, df.picks_6mos, df.qty_6mos, df.LEGACY_CUSTOMER_CD):
    if p in warehouse_to_prod[w]:
        wp_to_sales[w,p].append(s)
        wp_to_costs[w,p].append(c)
        wp_to_picks[w,p].append(pk)
        wp_to_quantity[w,p].append(q)
        if cust not in wp_to_ncustomers[w,p]:
            wp_to_ncustomers[w,p].append(cust)

for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_sales[w,p] = sum(wp_to_sales[w,p])
        wp_to_costs[w,p] = sum(wp_to_costs[w,p])
        wp_to_picks[w,p] = sum(wp_to_picks[w,p])
        wp_to_quantity[w,p] = sum(wp_to_quantity[w,p])
        wp_to_ncustomers[w,p] = len(wp_to_ncustomers[w,p])


#Create a dictionary of warehouse & product -> coreflag "Y" or "N"
wp_to_coreflag = {}
for w,p,cf in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.core_item_flag):
    if p in warehouse_to_prod[w]:
        if cf == "Y":
            wp_to_coreflag[w,p] = 1
        if cf == "N":
            wp_to_coreflag[w,p] = 0


#Create a dictionary of warehouse & product -> margin%
wp_to_margin = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        s = wp_to_sales[w,p]
        c = wp_to_costs[w,p]
#         ##DATA HAS TO BE CLEANED SO THAT COSTS THAT ARE EQUAL TO 0 DO NOT EXIST
#         if s == 0 or c==0:
#             wp_to_margin[w,p] = 0
#         else:
        wp_to_margin[w,p] = 100*((s-c)/s)


In [58]:
#Create a Dictionary of a warehouse, product cd pair with its net OH in units, net OH $ as values, item poi days & DIOH
#Example:
#Warehouse 19 houses product 10012415 and it has 4 units in on hand inventory, $81.2 of on hand inventory,
#302.430555555556 of average poi in days & 38.5263664820611 of DIOH
#The dictionary will look like this: wp_to_oh[19, 10012415] = [4.0, 81.2, 302.430555555556, 38.5263664820611]
wp_to_stats = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_stats[w,p] = [0,0,0,0]

for w, p, oh_usd, oh, poi, DIOH in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.net_OH_usd.values, df.net_OH.values,
                           df.item_poi_days.values, df.DIOH.values):
    if p in warehouse_to_prod[w]:
        if wp_to_stats[w,p] == [0,0,0,0]:
            wp_to_stats[w,p] = [oh, oh_usd, poi, DIOH]

In [59]:
#Create a Dictionary of warehouse & product to turn and earn
wp_to_TE = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        #netOH in USD
        net_oh = wp_to_stats[w,p][1]
        #cost of goods sold in USD
        cogs = wp_to_costs[w,p]
        #margin percentage
        mp = wp_to_margin[w,p]
        if net_oh == 0:
            turn = 0
        else:
            turn = cogs/net_oh
        wp_to_TE[w,p] = turn * mp    

In [60]:
#Create a Dictionary of warehouse & product -> total profit
wp_to_profit = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_profit[w,p] = wp_to_sales[w,p] - wp_to_costs[w,p]

In [61]:
for w in warehouses:
    print(w,'|', len(warehouse_to_prod[w]))

19 | 673
50 | 0
73 | 126
74 | 67
75 | 608
77 | 329
78 | 37
81 | 494
82 | 267
83 | 79
84 | 643
85 | 327
87 | 136
89 | 789
41 | 105


In [62]:
#Vector Model of Data at the Warehouse level

#Which percentage should we mark as core?
cutoff = 0.20
#Which warehouse do you want to see results for?
warehouse_to_view = 81

#Dictionary of warehouse & Prod -> vector model
wp_to_vector = {}


#Our model will take the distance of the vector formed by the TE, Total Profit and Number of customers
#Total Profit and TE can have both positive and negative values,
#because our model will square our values and then rank by vector size, a TE of -3000 will be considered
#to be better than a TE of 300, even though the TE of 300 is better. 

#we must rescale the wp_to_TE & wp_to_profit dictionaries
wp_to_TE_scaled = {}
wp_to_profit_scaled = {}
minimum_TE = np.absolute(min(wp_to_TE.values()))
minimum_profit = np.absolute(min(wp_to_profit.values()))

for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_TE_scaled[w,p] = wp_to_TE[w,p] + minimum_TE
        wp_to_profit_scaled[w,p] = wp_to_profit[w,p] + minimum_profit

#Create a dictionary of warehouse & prod to our flag
wp_to_flag = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_vector[w,p] = [wp_to_TE_scaled[w,p], wp_to_profit_scaled[w,p], wp_to_ncustomers[w,p]]
        wp_to_vector[w,p] = np.linalg.norm(wp_to_vector[w,p])


cutoff = 1- cutoff
for w in warehouses:
    prod_to_score = {}
    for p in warehouse_to_prod[w]:
        prod_to_score[p] = wp_to_vector[w,p]
    #sort products inside of a warehouse by their score in ascending order
    prods_by_score = sorted(prod_to_score, key=prod_to_score.__getitem__)
    cutoffIdx = int(len(prods_by_score)*cutoff)
    #the bottom cutoff% of the warehouse & product combination sorted by 
    non_core_prods = prods_by_score[:cutoffIdx]
    core_prods = prods_by_score[cutoffIdx:]
    for p in non_core_prods:
        wp_to_flag[w,p] = 0
    for p in core_prods:
        wp_to_flag[w,p] = 1

#Result statistics
#core products
core = []
#non core products
non_core = []

for p in warehouse_to_prod[warehouse_to_view]:
    if wp_to_flag[warehouse_to_view,p] == 0:
        non_core.append(p)
    else:
        core.append(p)

#Average profit
core_avg_profit = []
non_core_avg_profit = []
#Average turn and Earn
core_avg_TE = []
non_core_avg_TE = []
#Average number of customers
core_avg_ncust = []
non_core_avg_ncust = []

for p in core:
    core_avg_profit.append(wp_to_profit[warehouse_to_view,p])
    core_avg_TE.append(wp_to_TE[warehouse_to_view,p])
    core_avg_ncust.append(wp_to_ncustomers[warehouse_to_view,p])
core_avg_profit = np.average(core_avg_profit)
core_avg_TE = np.average(core_avg_TE)
core_avg_ncust = np.average(core_avg_ncust)

for p in non_core:
    non_core_avg_profit.append(wp_to_profit[warehouse_to_view,p])
    non_core_avg_TE.append(wp_to_TE[warehouse_to_view,p])
    non_core_avg_ncust.append(wp_to_ncustomers[warehouse_to_view,p])
non_core_avg_profit = np.average(non_core_avg_profit)
non_core_avg_TE = np.average(non_core_avg_TE)
non_core_avg_ncust = np.average(non_core_avg_ncust)


#Average profit
avg_profit = []
#Average TE
avg_TE = []
#Average number of customers
avg_ncust = []
for p in warehouse_to_prod[warehouse_to_view]:
    avg_profit.append(wp_to_profit[warehouse_to_view,p])
    avg_TE.append(wp_to_TE[warehouse_to_view, p])
    avg_ncust.append(wp_to_ncustomers[warehouse_to_view, p])
avg_profit = np.average(avg_profit)
avg_TE = np.average(avg_TE)
avg_ncust = np.average(avg_ncust)
    

print('For warehouse', warehouse_to_view)
print()
print("Number of core items", len(core))
print("Core Items Average Profit", core_avg_profit)
print("Core Items Average TE", core_avg_TE)
print('Core Items Average number of customers', core_avg_ncust)
print()
print("Number of non core items", len(non_core))
print("Non Core Items Average Profit", non_core_avg_profit)
print('Non Core Items Average TE', non_core_avg_TE)
print("Non Core Items Average number of customers", non_core_avg_ncust)
print()
print("Number of items:", len(core) + len(non_core))
print('All Items in warehouse average profit', avg_profit)
print('All Items in warehouse average TE', avg_TE)
print('All Items in warehouse average number of customers', avg_ncust)


For warehouse 81

Number of core items 99
Core Items Average Profit 25219.957004497643
Core Items Average TE 325.21004147843337
Core Items Average number of customers 2.212121212121212

Number of non core items 395
Non Core Items Average Profit 2465.839323783203
Non Core Items Average TE 174.7975528234101
Non Core Items Average number of customers 1.5924050632911393

Number of items: 494
All Items in warehouse average profit 7025.875053319092
All Items in warehouse average TE 204.9409462988095
All Items in warehouse average number of customers 1.7165991902834008


In [63]:
#Vector model at the HUB level

#Preparation

#Create a dictionary of region & product -> Total Profit
rp_to_profit = {}
#sales in USD
rp_to_sales = {}
#cost of goods sold in USD
rp_to_cogs = {}

#Create a dictionary of region & product -> Total Number of customers
rp_to_customers = {}
rp_to_ncustomers = {}

#Create a dictionary of region & product -> Turn and Earn
rp_to_TE = {}
#netOH in USD 
rp_to_netOH = {}

#turn = cogs/net_oh
#TE = turn * mp

#Create a region to products dictionary 
region_to_products = {}

regions = df.legacy_system_cd.unique()
for r in regions:
    region_to_products[r] = []

for r, p, sc, sales, costs, qsold, nOH, nOHusd in zip(df.legacy_system_cd, df.legacy_product_cd, df.sales_channel, 
                                         df.Sales_6_mos, df.cogs_6mos, df.qty_6mos, df.net_OH, df.net_OH_usd):
    if p not in region_to_products[r] and sc == "Warehouse" and sales > 0 and costs > 0 and qsold > 0 and nOH > 0 and nOHusd > 0:
        region_to_products[r].append(p)

for r in regions:
    for p in region_to_products[r]:
        rp_to_sales[r,p] = 0
        rp_to_cogs[r,p] = 0
        rp_to_customers[r,p] = []
        rp_to_netOH[r,p] = 0
        
for region, product, sale, cogs, cust, netOH_USD in zip(df.legacy_system_cd, df.legacy_product_cd, 
                                        df.Sales_6_mos, df.cogs_6mos, df.LEGACY_CUSTOMER_CD, df.net_OH_usd):
    if product in region_to_products[region]:
        rp_to_sales[region, product] += sale
        rp_to_cogs[region, product] += cogs
        if cust not in rp_to_customers[region,product]:
            rp_to_customers[region,product].append(cust)
        rp_to_netOH[region, product] += netOH_USD

for r in regions:
    for p in region_to_products[r]:
        sales = rp_to_sales[r,p]
        cogs = rp_to_cogs[r,p]
        rp_to_profit[r,p] = sales - cogs
        
        rp_to_ncustomers[r,p] = len(rp_to_customers[r,p])
        
        mp = 100*((sales-cogs)/sales)
        netOHusd = rp_to_netOH[r,p]
        if netOHusd == 0:
            turn = 0
        else:
            turn = cogs/netOHusd
        rp_to_TE[r,p] = turn * mp

In [64]:
#Vector Model of Data

#Which percentage should we mark as core?
cutoff = 0.20
#Which region do you want to see results for?
region_to_view = 'NU'


#Dictionary of region & Prod -> vector model
rp_to_vector = {}
#Create a dictionary of region & prod to our flag
rp_to_flag = {}


#Our model will take the distance of the vector formed by the TE, Total Profit and Number of customers
#Total Profit and TE can have both positive and negative values,
#because our model will square our values and then rank by vector size, a TE of -3000 will be considered
#to be better than a TE of 300, even though the TE of 300 is better. 

#we must rescale the rp_to_TE & rp_to_profit dictionaries
rp_to_TE_scaled = {}
rp_to_profit_scaled = {}
minimum_TE = np.absolute(min(rp_to_TE.values()))
minimum_profit = np.absolute(min(rp_to_profit.values()))

for r in regions:
    for p in region_to_products[r]:
        rp_to_TE_scaled[r,p] = rp_to_TE[r,p] + minimum_TE
        rp_to_profit_scaled[r,p] = rp_to_profit[r,p] + minimum_profit


for r in regions:
    for p in region_to_products[r]:
        rp_to_vector[r,p] = [rp_to_TE_scaled[r,p], rp_to_profit_scaled[r,p], rp_to_ncustomers[r,p]]
        rp_to_vector[r,p] = np.linalg.norm(rp_to_vector[r,p])


cutoff = 1- cutoff
for r in regions:
    prod_to_score = {}
    for p in region_to_products[r]:
        prod_to_score[p] = rp_to_vector[r,p]
    #sort products inside of a warehouse by their score in ascending order
    prods_by_score = sorted(prod_to_score, key=prod_to_score.__getitem__)
    cutoffIdx = int(len(prods_by_score)*cutoff)
    #the bottom cutoff% of the warehouse & product combination sorted by 
    non_core_prods = prods_by_score[:cutoffIdx]
    core_prods = prods_by_score[cutoffIdx:]
    for p in non_core_prods:
        rp_to_flag[r,p] = 0
    for p in core_prods:
        rp_to_flag[r,p] = 1

#Result statistics
#core products
core = []
#non core products
non_core = []

for p in region_to_products[region_to_view]:
    if rp_to_flag[region_to_view,p] == 0:
        non_core.append(p)
    else:
        core.append(p)

#Average profit
core_avg_profit = []
non_core_avg_profit = []
#Average turn and Earn
core_avg_TE = []
non_core_avg_TE = []
#Average number of customers
core_avg_ncust = []
non_core_avg_ncust = []

for p in core:
    core_avg_profit.append(rp_to_profit[region_to_view,p])
    core_avg_TE.append(rp_to_TE[region_to_view,p])
    core_avg_ncust.append(rp_to_ncustomers[region_to_view,p])
core_avg_profit = np.average(core_avg_profit)
core_avg_TE = np.average(core_avg_TE)
core_avg_ncust = np.average(core_avg_ncust)

for p in non_core:
    non_core_avg_profit.append(rp_to_profit[region_to_view,p])
    non_core_avg_TE.append(rp_to_TE[region_to_view,p])
    non_core_avg_ncust.append(rp_to_ncustomers[region_to_view,p])
non_core_avg_profit = np.average(non_core_avg_profit)
non_core_avg_TE = np.average(non_core_avg_TE)
non_core_avg_ncust = np.average(non_core_avg_ncust)


#Average profit
avg_profit = []
#Average TE
avg_TE = []
#Average number of customers
avg_ncust = []
for p in region_to_products[region_to_view]:
    avg_profit.append(rp_to_profit[region_to_view,p])
    avg_TE.append(rp_to_TE[region_to_view, p])
    avg_ncust.append(rp_to_ncustomers[region_to_view, p])
avg_profit = np.average(avg_profit)
avg_TE = np.average(avg_TE)
avg_ncust = np.average(avg_ncust)
    

print('For hub:', region_to_view)
print()
print("Number of core items", len(core))
print("Core Items Average Profit", core_avg_profit)
print("Core Items Average TE", core_avg_TE)
print('Core Items Average number of customers', core_avg_ncust)
print()
print("Number of non core items", len(non_core))
print("Non Core Items Average Profit", non_core_avg_profit)
print('Non Core Items Average TE', non_core_avg_TE)
print("Non Core Items Average number of customers", non_core_avg_ncust)
print()
print("Total number of items:", len(core) + len(non_core))
print('All Items in hub average profit', avg_profit)
print('All Items in hub average TE', avg_TE)
print('All Items in hub average number of customers', avg_ncust)

For hub: NU

Number of core items 21
Core Items Average Profit 24021.2444079489
Core Items Average TE 170.8471792286084
Core Items Average number of customers 1.9523809523809523

Number of non core items 84
Non Core Items Average Profit 2525.183257723775
Non Core Items Average TE 124.26347356310879
Non Core Items Average number of customers 1.7142857142857142

Total number of items: 105
All Items in hub average profit 6824.3954877688
All Items in hub average TE 133.58021469620869
All Items in hub average number of customers 1.7619047619047619


In [65]:
#For North Central Region

#Which percentage should we mark as core?
cutoff = 0.20

products = []
for  p, sc, sales, costs, qsold, nOH, nOHusd in zip(df.legacy_product_cd, df.sales_channel, 
                                         df.Sales_6_mos, df.cogs_6mos, df.qty_6mos, df.net_OH, df.net_OH_usd):
    if p not in products and sc == "Warehouse" and sales > 0 and costs > 0 and qsold > 0 and nOH > 0 and nOHusd > 0:
        products.append(p)


#Create Dictionary of products -> Total Sales
prod_to_sales = {}
#Create Dictionary of products -> Total Costs of Goods Sold
prod_to_cogs = {}
#Create Dictionary of products -> Net OH Inventory in USD
prod_to_netOH = {}
#Create Dictionary of products -> customers
prod_to_customers = {}
#Create Dictionary of products -> number of customers
prod_to_ncustomers = {}
#Create a Dictionary of products -> Total Profit
prod_to_profit = {}
#Create a Dictioanry of products -> TE
prod_to_TE = {}

for p in products:
    prod_to_sales[p] = 0
    prod_to_cogs[p] = 0
    prod_to_netOH[p] = 0
    prod_to_customers[p] = []
    prod_to_ncustomers[p] = 0

for  p, sales, costs, nOHusd, cust in zip(df.legacy_product_cd,df.Sales_6_mos, df.cogs_6mos,  df.net_OH_usd, df.LEGACY_CUSTOMER_CD):
    if p in products:
        prod_to_sales[p] += sales
        prod_to_cogs[p] += costs
        prod_to_netOH[p] += nOHusd
        if cust not in prod_to_customers[p]:
            prod_to_customers[p].append(cust)


        
for p in products:
    prod_to_ncustomers[p] = len(prod_to_customers[p])
    
    sales = prod_to_sales[p]
    cogs = prod_to_cogs[p]
    prod_to_profit[p] = sales - cogs
    
    mp = 100*((sales-cogs)/sales)
    netOHusd = prod_to_netOH[p]
    if netOHusd == 0:
        turn = 0
    else:
        turn = cogs/netOHusd
    prod_to_TE[p] = turn * mp

#Rescale TE & Profit to eliminate effect of negative numbers
minimum_TE = np.absolute(min(prod_to_TE.values()))
minimum_profit = np.absolute(min(prod_to_profit.values()))

#Create a dictionary of prod to TE scaled
prod_to_TE_scaled = {}
#Create a dictionary of prod to profit scaled
prod_to_profit_scaled = {}

for p in products:
    prod_to_TE_scaled[p] = prod_to_TE[p] + minimum_TE
    prod_to_profit_scaled[p] = prod_to_profit[p] + minimum_profit
    
#Create a dictionary of vector model
prod_to_vector = {}

#Create a dictionary with output of model -> Core Flag "Y"==1 and "N"==0
prod_to_flag = {}

for p in products:
    prod_to_vector[p] = [prod_to_TE_scaled[p], prod_to_profit_scaled[p], prod_to_ncustomers[p]]
    prod_to_vector[p] = np.linalg.norm(prod_to_vector[p])
    

cutoff = 1- cutoff
prod_to_score = prod_to_vector
#sort products inside of a warehouse by their score in ascending order
prods_by_score = sorted(prod_to_score, key=prod_to_score.__getitem__)
cutoffIdx = int(len(prods_by_score)*cutoff)
#the bottom cutoff% of the warehouse & product combination sorted by 
non_core_prods = prods_by_score[:cutoffIdx]
core_prods = prods_by_score[cutoffIdx:]
for p in non_core_prods:
    prod_to_flag[p] = 0
for p in core_prods:
    prod_to_flag[p] = 1

#Result statistics
#core products
core = []
#non core products
non_core = []

for p in products:
    if prod_to_flag[p] == 0:
        non_core.append(p)
    else:
        core.append(p)

#Average profit
core_avg_profit = []
non_core_avg_profit = []
#Average turn and Earn
core_avg_TE = []
non_core_avg_TE = []
#Average number of customers
core_avg_ncust = []
non_core_avg_ncust = []

for p in core:
    core_avg_profit.append(prod_to_profit[p])
    core_avg_TE.append(prod_to_TE[p])
    core_avg_ncust.append(prod_to_ncustomers[p])
core_avg_profit = np.average(core_avg_profit)
core_avg_TE = np.average(core_avg_TE)
core_avg_ncust = np.average(core_avg_ncust)

for p in non_core:
    non_core_avg_profit.append(prod_to_profit[p])
    non_core_avg_TE.append(prod_to_TE[p])
    non_core_avg_ncust.append(prod_to_ncustomers[p])
non_core_avg_profit = np.average(non_core_avg_profit)
non_core_avg_TE = np.average(non_core_avg_TE)
non_core_avg_ncust = np.average(non_core_avg_ncust)


#Average profit
avg_profit = []
#Average TE
avg_TE = []
#Average number of customers
avg_ncust = []
for p in products:
    avg_profit.append(prod_to_profit[p])
    avg_TE.append(prod_to_TE[p])
    avg_ncust.append(prod_to_ncustomers[p])
avg_profit = np.average(avg_profit)
avg_TE = np.average(avg_TE)
avg_ncust = np.average(avg_ncust)
    

print('For North Central Region')
print()
print("Number of core items", len(core))
print("Core Items Average Profit", core_avg_profit)
print("Core Items Average TE", core_avg_TE)
print('Core Items Average number of customers', core_avg_ncust)
print()
print("Number of non core items", len(non_core))
print("Non Core Items Average Profit", non_core_avg_profit)
print('Non Core Items Average TE', non_core_avg_TE)
print("Non Core Items Average number of customers", non_core_avg_ncust)
print()
print("Total number of items:", len(core) + len(non_core))
print('All Items in hub average profit', avg_profit)
print('All Items in hub average TE', avg_TE)
print('All Items in hub average number of customers', avg_ncust)

For North Central Region

Number of core items 752
Core Items Average Profit 30692.4287413653
Core Items Average TE 3891.8303763406466
Core Items Average number of customers 4.781914893617022

Number of non core items 3006
Non Core Items Average Profit 2149.893272414397
Non Core Items Average TE 113.48949235099097
Non Core Items Average number of customers 1.8572854291417165

Total number of items: 3758
All Items in hub average profit 7861.43842213528
All Items in hub average TE 869.5598342243867
All Items in hub average number of customers 2.44252261841405


In [66]:
#We find the ones that are core at any level 
#Non Core are the ones that are not these

prod_to_flags_visual = {}
prod_to_flags = {}

#add all the warehouse level flags
for w in warehouses:
    for p in warehouse_to_prod[w]:
        if p not in prod_to_flags.keys():
            prod_to_flags_visual[p] = [("w", w, wp_to_flag[w,p])]
            prod_to_flags[p] = [wp_to_flag[w,p]]
        else:
            prod_to_flags_visual[p].append(("w", w, wp_to_flag[w,p]))
            prod_to_flags[p].append(wp_to_flag[w,p])
            
#add all the region warehouse level flags
for r in regions:
    for p in region_to_products[r]:
        if p not in prod_to_flags.keys():
            prod_to_flags_visual[p] = [("hub", r, rp_to_flag[r,p])]
            prod_to_flags[p] = [rp_to_flag[r,p]]
        else:
            prod_to_flags_visual[p].append(("hub", r, rp_to_flag[r,p]))
            prod_to_flags[p].append(rp_to_flag[r,p])
            

#add all the north central region level flags
for p in products:
    if p not in prod_to_flags.keys():
        prod_to_flags_visual[p] = [("nc", prod_to_flag[p])]
        prod_to_flags[p] = [prod_to_flag[p]]
    else:
        prod_to_flags_visual[p].append(("nc",prod_to_flag[p]))
        prod_to_flags[p].append(prod_to_flag[p])

all_products = list(prod_to_flags.keys())
print(len(all_products))
print(list(prod_to_flags_visual.items())[0])

3758
(10023283, [('w', 19, 0), ('hub', 'MA', 0), ('nc', 0)])


In [67]:
prod_to_final = {}
for p in all_products:
    prod_to_final[p] = sum(prod_to_flags[p])
#list of all the products that are non core at every level
final_non_core = []
for p in all_products:
    if prod_to_final[p] == 0:
        final_non_core.append(p)
print(len(final_non_core))

2853


In [68]:
#We have identified a list of product cds that are non core at Every level
#We will now score these non core items by their north central region performance
#we will utilize the north central region model

#For worst performing products

#Which bottom percentage would you attribute as non core?
cutoff_bottom = 0.2


cutoff = 1- cutoff_bottom
products = final_non_core


#Create Dictionary of products -> Total Sales
bad_prod_to_sales = {}
#Create Dictionary of products -> Total Costs of Goods Sold
bad_prod_to_cogs = {}
#Create Dictionary of products -> Net OH Inventory in USD
bad_prod_to_netOH = {}
#Create Dictionary of products -> customers
bad_prod_to_customers = {}
#Create Dictionary of products -> number of customers
bad_prod_to_ncustomers = {}
#Create a Dictionary of products -> Total Profit
bad_prod_to_profit = {}
#Create a Dictioanry of products -> TE
bad_prod_to_TE = {}

for p in products:
    bad_prod_to_sales[p] = 0
    bad_prod_to_cogs[p] = 0
    bad_prod_to_netOH[p] = 0
    bad_prod_to_customers[p] = []
    bad_prod_to_ncustomers[p] = 0

for  p, sales, costs, nOHusd, cust in zip(df.legacy_product_cd,df.Sales_6_mos, df.cogs_6mos,  df.net_OH_usd, df.LEGACY_CUSTOMER_CD):
    if p in products:
        bad_prod_to_sales[p] += sales
        bad_prod_to_cogs[p] += costs
        bad_prod_to_netOH[p] += nOHusd
        if cust not in bad_prod_to_customers[p]:
            bad_prod_to_customers[p].append(cust)


        
for p in products:
    bad_prod_to_ncustomers[p] = len(bad_prod_to_customers[p])
    
    sales = bad_prod_to_sales[p]
    cogs = bad_prod_to_cogs[p]
    bad_prod_to_profit[p] = sales - cogs
    
    mp = 100*((sales-cogs)/sales)
    netOHusd = bad_prod_to_netOH[p]
    if netOHusd == 0:
        turn = 0
    else:
        turn = cogs/netOHusd
    bad_prod_to_TE[p] = turn * mp

#Rescale TE & Profit to eliminate effect of negative numbers
bad_minimum_TE = np.absolute(min(prod_to_TE.values()))
bad_minimum_profit = np.absolute(min(prod_to_profit.values()))

#Create a dictionary of prod to TE scaled
bad_prod_to_TE_scaled = {}
#Create a dictionary of prod to profit scaled
bad_prod_to_profit_scaled = {}

for p in products:
    bad_prod_to_TE_scaled[p] = bad_prod_to_TE[p] + bad_minimum_TE
    bad_prod_to_profit_scaled[p] = bad_prod_to_profit[p] + bad_minimum_profit
    
#Create a dictionary of vector model
bad_prod_to_vector = {}

#Create a dictionary with output of model -> Core Flag "Y"==1 and "N"==0
bad_prod_to_flag = {}

for p in products:
    bad_prod_to_vector[p] = [bad_prod_to_TE_scaled[p], bad_prod_to_profit_scaled[p], bad_prod_to_ncustomers[p]]
    bad_prod_to_vector[p] = np.linalg.norm(bad_prod_to_vector[p])
    

cutoff = 1- cutoff
prod_to_score = bad_prod_to_vector
#sort products inside of a warehouse by their score in ascending order
prods_by_score = sorted(prod_to_score, key=prod_to_score.__getitem__)
cutoffIdx = int(len(prods_by_score)*cutoff)
#the bottom cutoff% of the warehouse & product combination sorted by 
non_core_prods = prods_by_score[:cutoffIdx]
core_prods = prods_by_score[cutoffIdx:]
for p in non_core_prods:
    prod_to_flag[p] = 0
for p in core_prods:
    prod_to_flag[p] = 1

#Result statistics
#core products
core = []
#non core products
non_core = []

for p in products:
    if prod_to_flag[p] == 0:
        non_core.append(p)
    else:
        core.append(p)

#Average profit
core_avg_profit = []
non_core_avg_profit = []
#Average turn and Earn
core_avg_TE = []
non_core_avg_TE = []
#Average number of customers
core_avg_ncust = []
non_core_avg_ncust = []

for p in core:
    core_avg_profit.append(prod_to_profit[p])
    core_avg_TE.append(prod_to_TE[p])
    core_avg_ncust.append(prod_to_ncustomers[p])
core_avg_profit = np.average(core_avg_profit)
core_avg_TE = np.average(core_avg_TE)
core_avg_ncust = np.average(core_avg_ncust)

for p in non_core:
    non_core_avg_profit.append(prod_to_profit[p])
    non_core_avg_TE.append(prod_to_TE[p])
    non_core_avg_ncust.append(prod_to_ncustomers[p])
non_core_avg_profit = np.average(non_core_avg_profit)
non_core_avg_TE = np.average(non_core_avg_TE)
non_core_avg_ncust = np.average(non_core_avg_ncust)


#Average profit
avg_profit = []
#Average TE
avg_TE = []
#Average number of customers
avg_ncust = []
for p in products:
    avg_profit.append(prod_to_profit[p])
    avg_TE.append(prod_to_TE[p])
    avg_ncust.append(prod_to_ncustomers[p])
avg_profit = np.average(avg_profit)
avg_TE = np.average(avg_TE)
avg_ncust = np.average(avg_ncust)
    

print('For the worst performing items at all level ')
print()
print("Number of items to remain", len(core))
print("Items to remain Average Profit", core_avg_profit)
print("Items to remain Average TE", core_avg_TE)
print('Items to remain Average number of customers', core_avg_ncust)
print()
print("Number of items to be cut", len(non_core))
print("Items to be cut Average Profit", non_core_avg_profit)
print('Items to be cut Average TE', non_core_avg_TE)
print("Items to be cut Average number of customers", non_core_avg_ncust)
print()
print("Total number of items:", len(core) + len(non_core))
print('All worst performing items average profit', avg_profit)
print('All worst performing items average TE', avg_TE)
print('All worst performing items average number of customers', avg_ncust)

For the worst performing items at all level 

Number of items to remain 2283
Items to remain Average Profit 2318.110390189618
Items to remain Average TE 127.0839566353813
Items to remain Average number of customers 2.0197109067017083

Number of items to be cut 570
Items to be cut Average Profit 116.9929285226448
Items to be cut Average TE 41.76842702212011
Items to be cut Average number of customers 1.2666666666666666

Total number of items: 2853
All worst performing items average profit 1878.3498037367
All worst performing items average TE 110.03879299025024
All worst performing items average number of customers 1.869260427620049


In [69]:
#Products that will be cut
to_be_cut_final = []
for p in products:
    if prod_to_flag[p] == 0:
        to_be_cut_final.append(p)


#add the average total profit per customer to the vector model?

#Average Profit of all products
profits_all_products = []
for p in all_products:
    profits_all_products.append(prod_to_profit[p])
avg_profit_all_products = np.average(profits_all_products)
total_profit_all_products = sum(profits_all_products)
#Average Profit without the cut products
profits_products_after_cut = []
for p in all_products:
    if p not in to_be_cut_final:
        profits_products_after_cut.append(prod_to_profit[p])
avg_profit_after_cut = np.average(profits_products_after_cut)
total_profit_after_cut = sum(profits_products_after_cut)
#On Hand Inventory in USD of all products
netOH_usd_all_products = []
for p in all_products:
    netOH_usd_all_products.append(prod_to_netOH[p])
netOH_usd_all_products = sum(netOH_usd_all_products)
#On hand Inventory in USD without the cut products
net_OH_usd_after_cut = []
for p in all_products:
    if p not in to_be_cut_final:
        net_OH_usd_after_cut.append(prod_to_netOH[p])
net_OH_usd_after_cut = sum(net_OH_usd_after_cut)


print("Average Profit of all products", avg_profit_all_products)
print("Average Profit of products after cut", avg_profit_after_cut)
print()
print("Net OH Inventory USD all products", netOH_usd_all_products)
print("Net OH Invetnory USD after cut", net_OH_usd_after_cut)
reduction_OH_inventory = netOH_usd_all_products-net_OH_usd_after_cut
print("Reduction in OH Inventory USD", reduction_OH_inventory)
print()
print("Total Profit of all Products", total_profit_all_products)
print("Total Profit after cut", total_profit_after_cut)
reduction_profit = total_profit_all_products- total_profit_after_cut
print("Reduction in profit", reduction_profit)
print()
print("Total increase in cashflow:")
print("Reduction in OH Inventory USD - Reduction in Profit=", reduction_OH_inventory-reduction_profit)

Average Profit of all products 7861.438422135281
Average Profit of products after cut 9246.110295209059

Net OH Inventory USD all products 38473840.61184298
Net OH Invetnory USD after cut 37502184.56388697
Reduction in OH Inventory USD 971656.0479560122

Total Profit of all Products 29543285.59038431
Total Profit after cut 29476599.621126406
Reduction in profit 66685.96925790235

Total increase in cashflow:
Reduction in OH Inventory USD - Reduction in Profit= 904970.0786981098
